In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/unsupervised-ml-predict/sample_submission.csv
/kaggle/input/unsupervised-ml-predict/movies.csv
/kaggle/input/unsupervised-ml-predict/imdb_data.csv
/kaggle/input/unsupervised-ml-predict/genome_tags.csv
/kaggle/input/unsupervised-ml-predict/train.csv
/kaggle/input/unsupervised-ml-predict/test.csv
/kaggle/input/unsupervised-ml-predict/tags.csv
/kaggle/input/unsupervised-ml-predict/links.csv


In [4]:
# Load data
train = pd.read_csv('/kaggle/input/unsupervised-ml-predict/train.csv')
test = pd.read_csv('/kaggle/input/unsupervised-ml-predict/test.csv')

# Preprocess data
# Merge train set and movies set on movieId
merged = pd.merge(train, movies, on='movieId')

# Extract year from title and create a new feature
merged['year'] = merged['title'].str.extract('(\(\d\d\d\d\))',expand=False)
merged['year'] = merged['year'].str.extract('(\d\d\d\d)',expand=False)
merged['year'] = merged['year'].fillna(0)
merged['year'] = merged['year'].astype(int)

# Removing the years from the 'title' column
merged['title'] = merged['title'].str.replace('(\(\d\d\d\d\))', '')
merged['title'] = merged['title'].apply(lambda x: x.strip())

# Convert genres into one-hot vectors
merged['genres'] = merged['genres'].str.split('|')
merged = merged.join(pd.get_dummies(merged.genres.apply(pd.Series).stack()).sum(level=0))

# Split data into train and validation
X = merged.drop(['userId', 'movieId', 'rating', 'title', 'genres'], axis=1)
y = merged['rating']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on validation set and calculate RMSE
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print('RMSE:', rmse)

# Prepare test set
test_merged = pd.merge(test, movies, on='movieId')
test_merged['year'] = test_merged['title'].str.extract('(\(\d\d\d\d\))',expand=False)
test_merged['year'] = test_merged['year'].str.extract('(\d\d\d\d)',expand=False)
test_merged['year'] = test_merged['year'].fillna(0)
test_merged['year'] = test_merged['year'].astype(int)
test_merged['title'] = test_merged['title'].str.replace('(\(\d\d\d\d\))', '')
test_merged['title'] = test_merged['title'].apply(lambda x: x.strip())
test_merged['genres'] = test_merged['genres'].str.split('|')
test_merged = test_merged.join(pd.get_dummies(test_merged.genres.apply(pd.Series).stack()).sum(level=0))
X_test = test_merged.drop(['userId', 'movieId', 'title', 'genres'], axis=1)

# Predict on test set
y_test_pred = model.predict(X_test)

# Prepare submission
submission = pd.DataFrame({
    'Id': test['userId'].astype(str) + "_" + test['movieId'].astype(str),
    'rating': y_test_pred
})
submission.to_csv('submission.csv', index=False)

NameError: name 'movies' is not defined